In [1]:
import os
import time
import datetime
import re
import shutil
from collections import deque
import argparse

import numpy as np
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from simulation.simulations.data_generator import DataGenerator
from TransformerMOT.util.misc import save_checkpoint, update_logs
from TransformerMOT.util.load_config_files import load_yaml_into_dotdict
from TransformerMOT.util.plotting import output_truth_plot, compute_avg_certainty, get_constrastive_ax, get_false_ax, \
    get_total_loss_ax, get_state_uncertainties_ax
from TransformerMOT.util.logger import Logger
from TransformerMOT.models.BOMTv2 import BOMT
from simulation.simulations.data_generator import DataGenerator, get_single_training_example


In [2]:
task_params = r"C:\Users\chiny\OneDrive - Nanyang Technological University\Y3S2 (Internship)\MultiTracking\configs\tasks\task2.yaml"
model_params = r"C:\Users\chiny\OneDrive - Nanyang Technological University\Y3S2 (Internship)\MultiTracking\configs\models\BOMTv2.yaml"

params = load_yaml_into_dotdict(task_params)
params.update(load_yaml_into_dotdict(model_params))

if params.general.pytorch_and_numpy_seed is None:
    random_data = os.urandom(4)
    params.general.pytorch_and_numpy_seed = int.from_bytes(random_data, byteorder="big")
print(f'Using seed: {params.general.pytorch_and_numpy_seed}')

if params.training.device == 'auto':
    params.training.device = 'cuda' if torch.cuda.is_available() else 'cpu'

Using seed: 3370639121


In [3]:
from TransformerMOT.util.misc import NestedTensor

data_generator = DataGenerator(params=params)
training_nested_tensor, labels, unique_measurement_ids, target_coordinates = data_generator.get_batch()



In [4]:
model1 = BOMT(params).to("cuda")
# checkpoint = torch.load(r"results\train2_2phases_1\checkpoints\checkpoint_gradient_step_1615.pth",map_location="cuda")
# model1.load_state_dict(checkpoint["model_state_dict"])
res = model1(training_nested_tensor.to("cuda"))

In [5]:
res = model1.forward(
                measurements=training_nested_tensor.to("cuda"),
                target_coordinates=target_coordinates.to("cuda"),
                unique_id=unique_measurement_ids.to("cuda"),
            )

In [6]:
res[-1].shape

torch.Size([2, 1504])